In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.ensemble import IsolationForest, AdaBoostClassifier, BaggingClassifier, BaseEnsemble, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from catboost import CatBoostClassifier
import eli5
import shap
from sklearn import model_selection
from imblearn.under_sampling import TomekLinks
import umap

pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [2]:
train = pd.read_csv('../data/CAX_MortgageModeling_Train.csv')
train_len = train.shape[0]

In [3]:
data = pd.read_csv('../data/data_full_categorical_1705.csv')

In [4]:
cnt_cols = []
cat_num_cols = []
cat_cols = []
for col in data.columns:
    if data[col].dtype == object:
        cat_cols.append(col)
    else:
        if col != 'RESULT' and 'ordinal' not in col:
            cnt_cols.append(col)
        else:
            cat_num_cols.append(col)

In [5]:
cat_cols.remove('Unique_ID')
cat_num_cols.remove('RESULT')

In [6]:
cat_cols

['MORTGAGE PURPOSE',
 'PAYMENT FREQUENCY',
 'PROPERTY TYPE',
 'FSA',
 'GENDER',
 'NAICS CODE',
 'AGE RANGE_MORTGAGE PURPOSE_PROPERTY TYPE',
 'GENDER_MORTGAGE PURPOSE_PROPERTY TYPE',
 'FSA_MORTGAGE PURPOSE_PROPERTY TYPE',
 'NAICS CODE_MORTGAGE PURPOSE_PROPERTY TYPE']

In [7]:
cat_num_cols

['PROPERTY VALUE_ordinal',
 'MORTGAGE PAYMENT_ordinal',
 'GDS_ordinal',
 'LTV_ordinal',
 'TDS_ordinal',
 'MORTGAGE AMOUNT_ordinal',
 'RATE_ordinal',
 'AGE RANGE_ordinal',
 'INCOME_ordinal',
 'INCOME TYPE_ordinal',
 'CREDIT SCORE_ordinal',
 'APPRAISED PROPERTY VALUE_ordinal',
 'CHANGE IN PROPERTY VALUE_ordinal',
 'ANNUALIZED HOUSING EXPENSE_ordinal',
 'MORTGAGE RATIO_ordinal',
 'OTHER EXPENSE_ordinal',
 'RENT INCOME_ordinal',
 'ACTUAL MORTGAGE PAYMENT_ordinal',
 'TOTAL INTEREST_ordinal',
 'MORTGAGE AMOUNT PER INCOME_ordinal',
 'MORTGAGE PAYMENT PER INCOME_ordinal',
 'PROPERTY VALUE PER INCOME_ordinal',
 'TOTAL INTEREST PER INCOME_ordinal',
 'OTHER EXPENSE PER INCOME_ordinal',
 'ANNUALIZED HOUSING EXPENSE PER INCOME_ordinal',
 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE_ordinal',
 'REPAYMENT AGE_ordinal']

In [8]:
cnt_cols

['PROPERTY VALUE',
 'MORTGAGE PAYMENT',
 'GDS',
 'LTV',
 'TDS',
 'AMORTIZATION',
 'MORTGAGE AMOUNT',
 'RATE',
 'TERM',
 'AGE RANGE',
 'INCOME',
 'INCOME TYPE',
 'CREDIT SCORE',
 'APPRAISED PROPERTY VALUE',
 'CHANGE IN PROPERTY VALUE',
 'ANNUALIZED HOUSING EXPENSE',
 'MORTGAGE RATIO',
 'OTHER EXPENSE',
 'RENT INCOME',
 'PAYMENT FREQUENCY IN MONTHS',
 'ACTUAL MORTGAGE PAYMENT',
 'TOTAL INTEREST',
 'MORTGAGE AMOUNT PER INCOME',
 'MORTGAGE PAYMENT PER INCOME',
 'PROPERTY VALUE PER INCOME',
 'TOTAL INTEREST PER INCOME',
 'OTHER EXPENSE PER INCOME',
 'ANNUALIZED HOUSING EXPENSE PER INCOME',
 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE',
 'Is unemployed',
 'Is recently married',
 'Is married',
 'Is established',
 'Is about to retire',
 'Is retired',
 'REPAYMENT AGE']

In [9]:
data_cnt = pd.read_csv('../data/data_full_final.csv')

In [10]:
for col in data_cnt.columns:
    if col not in list(data.columns) and data_cnt[col].dtype == float and data_cnt[col].nunique() > 2:
        data[col] = data_cnt[col].values
        cnt_cols.append(col)

In [11]:
print (len(cnt_cols))

476


In [12]:
for col in cnt_cols:
    if 'Z SCORE' in col or 'MEAN' in col:
        data[col+"_ordinal"] = data[col].rank(pct=True)*10
        data[col+"_ordinal"] = data[col+"_ordinal"].astype(int)
        cat_num_cols.append(col+"_ordinal")

In [13]:
train_embedding = pd.read_csv('../data/train_AE_embeddings.csv')
test_embedding = pd.read_csv('../data/test_AE_embeddings.csv')
data_embedding = pd.concat([train_embedding,test_embedding],axis=0)
del train_embedding, test_embedding

In [14]:
for col in data_embedding.columns:
    data[col] = data_embedding[col].values
    cnt_cols.append(col)

In [15]:
data.head(10)

Unique_ID  PROPERTY VALUE  MORTGAGE PAYMENT     GDS      LTV     TDS  \
0   CAX_Train_1          900000              5429  0.6198  0.65000  0.7163   
1   CAX_Train_2          386000              2179  0.3522  0.74290  0.4065   
2   CAX_Train_3          531000              2152  0.3097  0.80000  0.3541   
3   CAX_Train_4         1200000              5410  0.1904  0.75000  0.3414   
4   CAX_Train_5          350000              3342  0.2959  0.80000  0.3485   
5   CAX_Train_6          420000              2179  0.2616  0.69050  0.2616   
6   CAX_Train_7         1600000              5275  0.3212  0.69230  0.3390   
7   CAX_Train_8         1500000              4383  0.3568  0.55170  0.3781   
8   CAX_Train_9          790000              3092  0.3529  0.78481  0.3540   
9  CAX_Train_10          650000              1839  0.3878  0.41850  0.3898   

   AMORTIZATION  MORTGAGE AMOUNT   RATE MORTGAGE PURPOSE PAYMENT FREQUENCY  \
0           360        1040000.0  0.040        Refinance           Monthly   
1           360         390000.0  0.045        Refinance           Monthly   
2           360         424800.0  0.035         Purchase           Monthly   
3           360         960000.0  0.055         Purchase           Monthly   
4           360         592000.0  0.050        Refinance           Monthly   
5           240         290000.0  0.055        Refinance           Monthly   
6           360         900000.0  0.040        Refinance           Monthly   
7           360         644000.0  0.040        Refinance           Monthly   
8           360         620000.0  0.035        Refinance           Monthly   
9           360         272000.0  0.055         Purchase           Monthly   

     PROPERTY TYPE  TERM FSA  AGE RANGE  GENDER  INCOME  INCOME TYPE  \
0  Single Detached    12  L4         25    Male  108000            8   
1  Single Detached    12  L9         75    Male   78000            2   
2  Single Detached     6  M1         39  Female   87000            2   
3  Single Detached    12  M2         49    Male  300000            8   
4  Single Detached    12  L7         54    Male  147000            8   
5  Single Detached    24  M1         44  Female  106000            2   
6  Single Detached    12  L7         59  Female  185000            8   
7  Single Detached    12  L3         49    Male  160000            8   
8  Single Detached    60  M4         49  Female   89000            2   
9  Single Detached    12  L4         44    Male   60000            2   

  NAICS CODE  CREDIT SCORE  RESULT  APPRAISED PROPERTY VALUE  \
0      44-45           681     0.0              1.600000e+06   
1         56           710     0.0              5.249697e+05   
2         72           709     0.0              5.310000e+05   
3         54           761     0.0              1.280000e+06   
4         62           762     0.0              7.400000e+05   
5         53           570     0.0              4.199855e+05   
6         56           812     0.0              1.300014e+06   
7      48-49           683     0.0              1.167301e+06   
8         61           684     1.0              7.900001e+05   
9         56           716     0.0              6.499403e+05   

   CHANGE IN PROPERTY VALUE  ANNUALIZED HOUSING EXPENSE  MORTGAGE RATIO  \
0              7.777778e-01                     66938.4        1.155556   
1              3.600252e-01                     27471.6        1.010363   
2              0.000000e+00                     26943.9        0.800000   
3              6.666667e-02                     57120.0        0.800000   
4              1.114286e+00                     43497.3        1.691429   
5             -3.448157e-05                     27729.6        0.690476   
6             -1.874910e-01                     59422.0        0.562500   
7             -2.217993e-01                     57088.0        0.429333   
8              1.612903e-07                     31408.1        0.784810   
9             -9.190332e-05             

In [16]:
train = data.iloc[:train_len]
test = data.iloc[train_len:]

In [17]:
train.shape, test.shape

((45642, 813), (15214, 813))

In [18]:
train.RESULT.value_counts(normalize=True)

0.0    0.789251
1.0    0.210749
Name: RESULT, dtype: float64

In [19]:
train2 = pd.concat([train[train.RESULT == 0].sample(frac=.26),train[train.RESULT == 1]],axis=0).sample(frac=1)

In [20]:
kf = KFold(n_splits=5,random_state=123,shuffle=True)
for train_index, test_index in kf.split(train):
    break
for train_index2, test_index2 in kf.split(train2):
    break

In [21]:
train2.shape

(18985, 813)

In [22]:
train2[['RESULT']+cat_cols+cat_num_cols+cnt_cols].head(10)

RESULT MORTGAGE PURPOSE PAYMENT FREQUENCY    PROPERTY TYPE FSA  GENDER  \
3381      0.0        Refinance           Monthly  Single Detached  N5    Male   
2000      0.0         Purchase           Monthly        High Rise  M6  Female   
22714     1.0        Refinance           Monthly  Single Detached  L6  Female   
25025     1.0         Purchase           Monthly  Single Detached  M5    Male   
44496     0.0         Purchase           Monthly        High Rise  L5    Male   
23654     1.0         Purchase           Monthly  Single Detached  V8    Male   
10319     1.0         Purchase           Monthly        High Rise  M2    Male   
45622     0.0        Refinance           Monthly  Single Detached  L5    Male   
38172     1.0         Purchase           Monthly  Single Detached  M2  Female   
5159      1.0        Refinance           Monthly  Single Detached  K2    Male   

      NAICS CODE AGE RANGE_MORTGAGE PURPOSE_PROPERTY TYPE  \
3381          81             69_Refinance_Single Detached   
2000       44-45                    59_Purchase_High Rise   
22714         81             39_Refinance_Single Detached   
25025         54              54_Purchase_Single Detached   
44496      44-45                    34_Purchase_High Rise   
23654         23              69_Purchase_Single Detached   
10319         61                    49_Purchase_High Rise   
45622         56             59_Refinance_Single Detached   
38172         23              39_Purchase_Single Detached   
5159          23             54_Refinance_Single Detached   

      GENDER_MORTGAGE PURPOSE_PROPERTY TYPE  \
3381         Male_Refinance_Single Detached   
2000              Female_Purchase_High Rise   
22714      Female_Refinance_Single Detached   
25025         Male_Purchase_Single Detached   
44496               Male_Purchase_High Rise   
23654         Male_Purchase_Single Detached   
10319               Male_Purchase_High Rise   
45622        Male_Refinance_Single Detached   
38172       Female_Purchase_Single Detached   
5159         Male_Refinance_Single Detached   

      FSA_MORTGAGE PURPOSE_PROPERTY TYPE  \
3381        N5_Refinance_Single Detached   
2000               M6_Purchase_High Rise   
22714       L6_Refinance_Single Detached   
25025        M5_Purchase_Single Detached   
44496              L5_Purchase_High Rise   
23654        V8_Purchase_Single Detached   
10319              M2_Purchase_High Rise   
45622       L5_Refinance_Single Detached   
38172        M2_Purchase_Single Detached   
5159        K2_Refinance_Single Detached   

      NAICS CODE_MORTGAGE PURPOSE_PROPERTY TYPE  PROPERTY VALUE_ordinal  \
3381               81_Refinance_Single Detached                       0   
2000                   44-45_Purchase_High Rise                       3   
22714              81_Refinance_Single Detached                       8   
25025               54_Purchase_Single Detached                       8   
44496                  44-45_Purchase_High Rise                       3   
23654               23_Purchase_Single Detached                       9   
10319                     61_Purchase_High Rise                       2   
45622              56_Refinance_Single Detached                       7   
38172               23_Purchase_Single Detached                       9   
5159               23_Refinance_Single Detached                       7   

       MORTGAGE PAYMENT_ordinal  GDS_ordinal  LTV_ordinal  TDS_ordinal  \
3381                          0            0            2            0   
2000                          2            0            5            7   
22714                         8            8            3            6   
25025                         8            7            5            6   
44496                         3            0            8            1   
23654                         9            5            2            9   
10319                         2            6            5            8   
45622                 

In [23]:
model = CatBoostClassifier(iterations=500,learning_rate=0.25, loss_function= 'Logloss', eval_metric='F1',use_best_model=True,random_seed=42,thread_count=2)

In [24]:
model.fit(train2[cat_cols+cat_num_cols+cnt_cols].iloc[train_index2],train2.RESULT.iloc[train_index2],cat_features=np.arange(len(cat_cols+cat_num_cols)),eval_set=(train2[cat_cols+cat_num_cols+cnt_cols].iloc[test_index2],train2.RESULT.iloc[test_index2]))

0:	learn: 0.6426312	test: 0.6308345	best: 0.6308345 (0)	total: 1.7s	remaining: 14m 9s
1:	learn: 0.6522554	test: 0.6419639	best: 0.6419639 (1)	total: 3.32s	remaining: 13m 46s
2:	learn: 0.6455712	test: 0.6344929	best: 0.6419639 (1)	total: 6.35s	remaining: 17m 32s
3:	learn: 0.6467211	test: 0.6366366	best: 0.6419639 (1)	total: 9.41s	remaining: 19m 27s
4:	learn: 0.6556304	test: 0.6460396	best: 0.6460396 (4)	total: 12.3s	remaining: 20m 18s
5:	learn: 0.6474550	test: 0.6419125	best: 0.6460396 (4)	total: 14.6s	remaining: 19m 59s
6:	learn: 0.6556397	test: 0.6447566	best: 0.6460396 (4)	total: 17.3s	remaining: 20m 18s
7:	learn: 0.6562061	test: 0.6445283	best: 0.6460396 (4)	total: 19.3s	remaining: 19m 44s
8:	learn: 0.6553585	test: 0.6525360	best: 0.6525360 (8)	total: 22.4s	remaining: 20m 23s
9:	learn: 0.6580918	test: 0.6546185	best: 0.6546185 (9)	total: 24.5s	remaining: 20m 2s
10:	learn: 0.6591434	test: 0.6537591	best: 0.6546185 (9)	total: 27.5s	remaining: 20m 20s
11:	learn: 0.6629487	test: 0.65502

91:	learn: 0.7203562	test: 0.6761880	best: 0.6761880 (91)	total: 4m 52s	remaining: 21m 37s
92:	learn: 0.7202829	test: 0.6749870	best: 0.6761880 (91)	total: 4m 55s	remaining: 21m 33s
93:	learn: 0.7215820	test: 0.6748179	best: 0.6761880 (91)	total: 4m 58s	remaining: 21m 29s
94:	learn: 0.7218539	test: 0.6744975	best: 0.6761880 (91)	total: 5m 1s	remaining: 21m 26s
95:	learn: 0.7221018	test: 0.6767545	best: 0.6767545 (95)	total: 5m 4s	remaining: 21m 21s
96:	learn: 0.7230355	test: 0.6781519	best: 0.6781519 (96)	total: 5m 7s	remaining: 21m 18s
97:	learn: 0.7226086	test: 0.6781429	best: 0.6781519 (96)	total: 5m 10s	remaining: 21m 15s
98:	learn: 0.7236359	test: 0.6779573	best: 0.6781519 (96)	total: 5m 14s	remaining: 21m 12s
99:	learn: 0.7238933	test: 0.6776385	best: 0.6781519 (96)	total: 5m 17s	remaining: 21m 9s
100:	learn: 0.7248868	test: 0.6764092	best: 0.6781519 (96)	total: 5m 20s	remaining: 21m 7s
101:	learn: 0.7258520	test: 0.6789995	best: 0.6789995 (101)	total: 5m 24s	remaining: 21m 4s
10

180:	learn: 0.7548561	test: 0.6774362	best: 0.6847034 (137)	total: 9m 52s	remaining: 17m 23s
181:	learn: 0.7547741	test: 0.6770915	best: 0.6847034 (137)	total: 9m 56s	remaining: 17m 21s
182:	learn: 0.7547070	test: 0.6774362	best: 0.6847034 (137)	total: 9m 59s	remaining: 17m 18s
183:	learn: 0.7552981	test: 0.6765625	best: 0.6847034 (137)	total: 10m 2s	remaining: 17m 15s
184:	learn: 0.7566820	test: 0.6758945	best: 0.6847034 (137)	total: 10m 6s	remaining: 17m 12s
185:	learn: 0.7566002	test: 0.6760637	best: 0.6847034 (137)	total: 10m 9s	remaining: 17m 8s
186:	learn: 0.7561056	test: 0.6784876	best: 0.6847034 (137)	total: 10m 11s	remaining: 17m 3s
187:	learn: 0.7561056	test: 0.6784876	best: 0.6847034 (137)	total: 10m 14s	remaining: 17m
188:	learn: 0.7561554	test: 0.6784876	best: 0.6847034 (137)	total: 10m 18s	remaining: 16m 57s
189:	learn: 0.7563047	test: 0.6762477	best: 0.6847034 (137)	total: 10m 20s	remaining: 16m 53s
190:	learn: 0.7567354	test: 0.6774615	best: 0.6847034 (137)	total: 10m 2

268:	learn: 0.7777265	test: 0.6799071	best: 0.6847034 (137)	total: 14m 41s	remaining: 12m 36s
269:	learn: 0.7779754	test: 0.6780186	best: 0.6847034 (137)	total: 14m 44s	remaining: 12m 33s
270:	learn: 0.7791763	test: 0.6783596	best: 0.6847034 (137)	total: 14m 47s	remaining: 12m 29s
271:	learn: 0.7792790	test: 0.6778437	best: 0.6847034 (137)	total: 14m 51s	remaining: 12m 27s
272:	learn: 0.7795359	test: 0.6769866	best: 0.6847034 (137)	total: 14m 55s	remaining: 12m 24s
273:	learn: 0.7800119	test: 0.6771613	best: 0.6847034 (137)	total: 14m 58s	remaining: 12m 20s
274:	learn: 0.7800119	test: 0.6771613	best: 0.6847034 (137)	total: 15m 1s	remaining: 12m 17s
275:	learn: 0.7795447	test: 0.6782025	best: 0.6847034 (137)	total: 15m 4s	remaining: 12m 14s
276:	learn: 0.7799024	test: 0.6800826	best: 0.6847034 (137)	total: 15m 7s	remaining: 12m 10s
277:	learn: 0.7799314	test: 0.6797419	best: 0.6847034 (137)	total: 15m 10s	remaining: 12m 7s
278:	learn: 0.7799314	test: 0.6797419	best: 0.6847034 (137)	tota

357:	learn: 0.7931012	test: 0.6759474	best: 0.6847034 (137)	total: 19m 40s	remaining: 7m 48s
358:	learn: 0.7938138	test: 0.6749162	best: 0.6847034 (137)	total: 19m 43s	remaining: 7m 44s
359:	learn: 0.7938117	test: 0.6776604	best: 0.6847034 (137)	total: 19m 47s	remaining: 7m 41s
360:	learn: 0.7941273	test: 0.6776604	best: 0.6847034 (137)	total: 19m 51s	remaining: 7m 38s
361:	learn: 0.7943636	test: 0.6776604	best: 0.6847034 (137)	total: 19m 55s	remaining: 7m 35s
362:	learn: 0.7942786	test: 0.6786818	best: 0.6847034 (137)	total: 19m 58s	remaining: 7m 32s
363:	learn: 0.7936236	test: 0.6769706	best: 0.6847034 (137)	total: 20m 1s	remaining: 7m 29s
364:	learn: 0.7943870	test: 0.6771451	best: 0.6847034 (137)	total: 20m 5s	remaining: 7m 25s
365:	learn: 0.7954935	test: 0.6762812	best: 0.6847034 (137)	total: 20m 9s	remaining: 7m 22s
366:	learn: 0.7967587	test: 0.6754454	best: 0.6847034 (137)	total: 20m 13s	remaining: 7m 19s
367:	learn: 0.7967062	test: 0.6761364	best: 0.6847034 (137)	total: 20m 17

446:	learn: 0.8067935	test: 0.6744006	best: 0.6847034 (137)	total: 24m 36s	remaining: 2m 55s
447:	learn: 0.8069252	test: 0.6750968	best: 0.6847034 (137)	total: 24m 39s	remaining: 2m 51s
448:	learn: 0.8069252	test: 0.6750968	best: 0.6847034 (137)	total: 24m 42s	remaining: 2m 48s
449:	learn: 0.8069506	test: 0.6759402	best: 0.6847034 (137)	total: 24m 46s	remaining: 2m 45s
450:	learn: 0.8068975	test: 0.6762812	best: 0.6847034 (137)	total: 24m 49s	remaining: 2m 41s
451:	learn: 0.8075606	test: 0.6766452	best: 0.6847034 (137)	total: 24m 52s	remaining: 2m 38s
452:	learn: 0.8075606	test: 0.6756060	best: 0.6847034 (137)	total: 24m 55s	remaining: 2m 35s
453:	learn: 0.8081633	test: 0.6755991	best: 0.6847034 (137)	total: 24m 58s	remaining: 2m 31s
454:	learn: 0.8084490	test: 0.6752710	best: 0.6847034 (137)	total: 25m 1s	remaining: 2m 28s
455:	learn: 0.8084742	test: 0.6747548	best: 0.6847034 (137)	total: 25m 4s	remaining: 2m 25s
456:	learn: 0.8090036	test: 0.6740645	best: 0.6847034 (137)	total: 25m 8

In [53]:
#model = LGBMClassifier(n_estimators=100,random_state=42,n_jobs=2)

In [54]:
#model.fit(train2[cat_num_cols+cnt_cols].iloc[train_index2],train2.RESULT.iloc[train_index2],categorical_feature=cat_num_cols,eval_set=(train2[cat_num_cols+cnt_cols].iloc[test_index2],train2.RESULT.iloc[test_index2]),eval_metric='F1')

C:\Users\asengup6\Softwares\Anaconda\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['ACTUAL MORTGAGE PAYMENT AGE RANGE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE AGE RANGE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE FSA Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE GENDER Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE MORTGAGE PURPOSE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE NAICS CODE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE PAYMENT FREQUENCY Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE PROPERTY TYPE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT BY MORTGAGE_ordinal', 'ACTUAL MORTGAGE PAYMENT FSA Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT GENDER Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT MORTGAGE PURPOSE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT NAICS CODE Z SCORE_ordinal', 'ACTUAL MORTGAGE PAYMENT PAYMENT FREQUENCY Z SCORE_ordina

[1]	valid_0's binary_logloss: 0.673288
[2]	valid_0's binary_logloss: 0.664008
[3]	valid_0's binary_logloss: 0.655665
[4]	valid_0's binary_logloss: 0.650101
[5]	valid_0's binary_logloss: 0.64449
[6]	valid_0's binary_logloss: 0.639095
[7]	valid_0's binary_logloss: 0.632895
[8]	valid_0's binary_logloss: 0.628568
[9]	valid_0's binary_logloss: 0.624904
[10]	valid_0's binary_logloss: 0.620882
[11]	valid_0's binary_logloss: 0.618417
[12]	valid_0's binary_logloss: 0.616129
[13]	valid_0's binary_logloss: 0.613452
[14]	valid_0's binary_logloss: 0.612179
[15]	valid_0's binary_logloss: 0.610598
[16]	valid_0's binary_logloss: 0.609217
[17]	valid_0's binary_logloss: 0.607704
[18]	valid_0's binary_logloss: 0.605394
[19]	valid_0's binary_logloss: 0.604556
[20]	valid_0's binary_logloss: 0.604008
[21]	valid_0's binary_logloss: 0.603237
[22]	valid_0's binary_logloss: 0.60194
[23]	valid_0's binary_logloss: 0.601856
[24]	valid_0's binary_logloss: 0.60112
[25]	valid_0's binary_logloss: 0.600051
[26]	valid_0

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=2, num_leaves=31, objective=None,
        random_state=42, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [25]:
pred = model.predict(train2[cat_cols+cat_num_cols+cnt_cols].iloc[test_index2])
print (accuracy_score(train2.RESULT.iloc[test_index2],pred))
print (confusion_matrix(train2.RESULT.iloc[test_index2],pred))
print (f1_score(train2.RESULT.iloc[test_index2],pred))

0.6808006320779563
[[1269  567]
 [ 645 1316]]
0.6847034339229969


In [25]:
testpred = model.predict(test[cat_cols+cat_num_cols+cnt_cols])
submission1 = pd.read_csv('../data/CAX_MortgageModeling_SubmissionFormat.csv')
submission1.Result_Predicted = testpred
submission1.Result_Predicted = submission1.Result_Predicted.apply(lambda x: "FUNDED" if x == 0 else "NOT FUNDED")
print (submission1.Result_Predicted.value_counts(normalize=True))
submission1.to_csv('../submissions/submission29.csv',index=False)

FUNDED        0.58203
NOT FUNDED    0.41797
Name: Result_Predicted, dtype: float64
